In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('data/fake_news_dataset.csv')
df

,title,text,date,source,author,category,label
0,Foreign Democrat final.,more tax development both store agreement lawy...,2023-03-10,NY Times,Paula George,Politics,real
1,To offer down resource great point.,probably guess western behind likely next inve...,2022-05-25,Fox News,Joseph Hill,Politics,fake
2,Himself church myself carry.,them identify forward present success risk sev...,2022-09-01,CNN,Julia Robinson,Business,fake
3,You unit its should.,phone which item yard Republican safe where po...,2023-02-07,Reuters,Mr. David Foster DDS,Science,fake
4,Billion believe employee summer how.,wonder myself fact difficult course forget exa...,2023-04-03,CNN,Austin Walker,Technology,fake
...,...,...,...,...,...,...,...
19995,House party born.,hit and television I change very our happy doo...,2024-12-04,BBC,Gary Miles,Entertainment,fake
19996,Though nation people maybe price box.,fear most meet rock even sea value design stan...,2024-05-26,Daily News,Maria Mcbride,Entertainment,real
19997,Yet exist with experience unit.,activity loss very provide eye west create wha...,2023-04-17,BBC,Kristen Franklin,Entertainment,real
19998,School wide itself item.,term point general common training watch respo...,2024-06-30,Reuters,David Wise,Health,fake


In [ ]:
df.isna().sum()

title          0
text           0
date           0
source      1000
author      1000
category       0
label          0
dtype: int64

In [ ]:
df['author'] = df['author'].fillna('unknown')
df['source'] = df['source'].fillna('unknown')

In [ ]:
df.isna().sum()

title       0
text        0
date        0
source      0
author      0
category    0
label       0
dtype: int64

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     20000 non-null  object
 1   text      20000 non-null  object
 2   date      20000 non-null  object
 3   source    20000 non-null  object
 4   author    20000 non-null  object
 5   category  20000 non-null  object
 6   label     20000 non-null  object
dtypes: object(7)
memory usage: 1.1+ MB


In [ ]:
df['date'] = pd.to_datetime(df['date'], errors='coerce')

In [ ]:
df['full_text'] = df['title'].str.strip() + ' ' + df['text'].str.strip()

In [ ]:
df

,title,text,date,source,author,category,label,full_text
0,Foreign Democrat final.,more tax development both store agreement lawy...,2023-03-10,NY Times,Paula George,Politics,real,Foreign Democrat final. more tax development b...
1,To offer down resource great point.,probably guess western behind likely next inve...,2022-05-25,Fox News,Joseph Hill,Politics,fake,To offer down resource great point. probably g...
2,Himself church myself carry.,them identify forward present success risk sev...,2022-09-01,CNN,Julia Robinson,Business,fake,Himself church myself carry. them identify for...
3,You unit its should.,phone which item yard Republican safe where po...,2023-02-07,Reuters,Mr. David Foster DDS,Science,fake,You unit its should. phone which item yard Rep...
4,Billion believe employee summer how.,wonder myself fact difficult course forget exa...,2023-04-03,CNN,Austin Walker,Technology,fake,Billion believe employee summer how. wonder my...
...,...,...,...,...,...,...,...,...
19995,House party born.,hit and television I change very our happy doo...,2024-12-04,BBC,Gary Miles,Entertainment,fake,House party born. hit and television I change ...
19996,Though nation people maybe price box.,fear most meet rock even sea value design stan...,2024-05-26,Daily News,Maria Mcbride,Entertainment,real,Though nation people maybe price box. fear mos...
19997,Yet exist with experience unit.,activity loss very provide eye west create wha...,2023-04-17,BBC,Kristen Franklin,Entertainment,real,Yet exist with experience unit. activity loss ...
19998,School wide itself item.,term point general common training watch respo...,2024-06-30,Reuters,David Wise,Health,fake,School wide itself item. term point general co...


In [ ]:
import spacy
import re
from tqdm.auto import tqdm

In [ ]:
tqdm.pandas()

In [ ]:
nlp = spacy.load("en_core_web_sm")
nlp.disable_pipes("ner", "parser")

['ner', 'parser']

In [ ]:
CUSTOM_STOP = {
    'mr', 'mrs', 'ms', 'build', 'leave', 'good', 'include', 'meet', 'feel', 'well', 'think', 'late', 'well', 'add',
    'hour', 'day', 'month', 'minute', 'set', 'late'
}

URL_RE = re.compile(r"http\S+|www\.\S+")
SPACE_RE = re.compile(r"\s+")

In [ ]:
for w in CUSTOM_STOP:
    nlp.vocab[w].is_stop = True # расширяем stop_words из библиотеки

In [ ]:
def normalize_text_en(s: str) -> str:
    s = "" if s is None else str(s)
    s = URL_RE.sub(" ", s)
    s = s.replace("\n", " ")
    s = SPACE_RE.sub(" ", s).strip()
    return s

In [ ]:
def spacy_lemmas_pipe(texts, batch_size=512):
    texts = [normalize_text_en(t) for t in texts]
    out = []

    for doc in tqdm(nlp.pipe(texts, batch_size=batch_size), total=len(texts)):
        lemmas = []
        for t in doc:
            if t.is_space or t.is_punct or t.like_num:
                continue
            if t.is_stop:
                continue

            lemma = t.lemma_.lower().strip()

            if lemma in {"-pron-", ""}:
                continue
            if len(lemma) < 3:
                continue

            lemmas.append(lemma)
        out.append(lemmas)

    return out

df["tokens"] = spacy_lemmas_pipe(df["full_text"].astype(str).tolist(), batch_size=512)

  0%|          | 0/20000 [00:00<?, ?it/s]

In [ ]:
df

,title,text,date,source,author,category,label,full_text,tokens
0,Foreign Democrat final.,more tax development both store agreement lawyer hear outside continue reach difference yeah figure your power fear identify there protect security great national nothing fast story why late nearly bit cost tough since question to power almost future young conference behind ahead building teach million box receive Mrs risk benefit month compare environment class imagine you vote community reason set once idea him answer many how purpose deep training game own true language garden of partner result face military discover discover data glass bed maintain test way development across top culture glass yes decision hope necessary as trade organization talk debate peace stay community development six wide write itself several fight teach billion for common fear we personal church establish store kind hundred debate hotel cut sister audience sound case that stay within information trouble be debate great themselves responsibility force people hundred bar miss others sometimes build room interesting however charge what especially north no especially us travel industry about including face ten behind black series place age soldier early trouble middle would along case what money significant sound song reason poor free want thank cultural range shoulder rest movie political fear hear past leader up edge professor determine law act change middle prove say notice travel open director argue economic seven game matter season,2023-03-10,NY Times,Paula George,Politics,real,Foreign Democrat final. more tax development both store agreement lawyer hear outside continue reach difference yeah figure your power fear identify there protect security great national nothing fast story why late nearly bit cost tough since question to power almost future young conference behind ahead building teach million box receive Mrs risk benefit month compare environment class imagine you vote community reason set once idea him answer many how purpose deep training game own true language garden of partner result face military discover discover data glass bed maintain test way development across top culture glass yes decision hope necessary as trade organization talk debate peace stay community development six wide write itself several fight teach billion for common fear we personal church establish store kind hundred debate hotel cut sister audience sound case that stay within information trouble be debate great themselves responsibility force people hundred bar miss others sometimes build room interesting however charge what especially north no especially us travel industry about including face ten behind black series place age soldier early trouble middle would along case what money significant sound song reason poor free want thank cultural range shoulder rest movie political fear hear past leader up edge professor determine law act change middle prove say notice travel open director argue economic seven game matter season,"[foreign, democrat, final, tax, development, store, agreement, lawyer, hear, outside, continue, reach, difference, yeah, figure, power, fear, identify, protect, security, great, national, fast, story, nearly, bit, cost, tough, question, power, future, young, conference, ahead, building, teach, box, receive, mrs, risk, benefit, compare, environment, class, imagine, vote, community, reason, idea, answer, purpose, deep, training, game, true, language, garden, partner, result, face, military, discover, discover, datum, glass, bed, maintain, test, way, development, culture, glass, yes, decision, hope, necessary, trade, organization, talk, debate, peace, stay, community, development, wide, write, fight, teach, common, fear, personal, church, establish, store, kind, debate, hotel, cut, sister, audience, ...]"
1,To offer down resource great point.,probably guess western behind likely next investment consumer range wrong exactly once attack shoulder movie partner 

In [ ]:
from gensim.corpora import Dictionary
from gensim.models import TfidfModel

In [ ]:
dictionary = Dictionary(df['tokens'])
dictionary.filter_extremes(no_below=5, no_above=0.5)
dictionary.compactify()

In [ ]:
bow_all = [dictionary.doc2bow(toks) for toks in df["tokens"]]

In [ ]:
tfidf_model = TfidfModel(bow_all, normalize=True)

In [ ]:
from collections import defaultdict

In [ ]:
def class_specific_tfidf(
    df,
    tokens_col,
    group_col,
    dictionary,
    tfidf_model,
    topn=10,
    min_docs=10
):
    rows = []

    for group_value, part in df.groupby(group_col):
        if len(part) < min_docs:
            continue

        acc = defaultdict(float)

        # TF считаем только внутри группы
        for toks in part[tokens_col]:
            bow = dictionary.doc2bow(toks)
            tfidf_doc = tfidf_model[bow]
            for term_id, w in tfidf_doc:
                acc[term_id] += w

        top = sorted(acc.items(), key=lambda x: x[1], reverse=True)[:topn]
        words = [dictionary[term_id] for term_id, _ in top]

        rows.append((group_value, words))

    return pd.DataFrame(rows, columns=[group_col, f"top{topn}_words"])


In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
top_category_distinct = class_specific_tfidf(
    df=df,
    tokens_col="tokens",
    group_col="category",
    dictionary=dictionary,
    tfidf_model=tfidf_model,
    topn=10,
    min_docs=20
)

top_category_distinct

,category,top10_words
0,Business,"[local, entire, listen, well, vote, president, remain, bear, pay, source]"
1,Entertainment,"[well, check, manage, season, door, evening, near, ability, sense, stand]"
2,Health,"[write, poor, available, long, grow, evidence, well, different, expect, reason]"
3,Politics,"[fire, well, small, memory, rate, way, history, summer, interesting, cup]"
4,Science,"[image, writer, garden, able, activity, well, certain, common, appear, degree]"
5,Sports,"[inside, politic, speak, goal, attention, major, environmental, town, owner, sell]"
6,Technology,"[well, usually, present, doctor, cultural, hit, sport, believe, speech, story]"


In [ ]:
top_label_distinct = class_specific_tfidf(
    df=df,
    tokens_col="tokens",
    group_col="label",
    dictionary=dictionary,
    tfidf_model=tfidf_model,
    topn=10,
    min_docs=20
)

display(top_label_distinct)


,label,top10_words
0,fake,"[well, moment, claim, significant, offer, wide, particular, rich, subject, summer]"
1,real,"[well, economy, reflect, remain, play, member, drive, produce, girl, bring]"


In [ ]:
fake_words = set(top_label_distinct[top_label_distinct["label"]=="fake"]["top10_words"].iloc[0])
real_words = set(top_label_distinct[top_label_distinct["label"]=="real"]["top10_words"].iloc[0])

print("Intersection fake & real:", fake_words & real_words)


Intersection fake & real: {'well'}


In [ ]:
author_stats = (
    df.groupby("author")
      .agg(
          n=("label", "size"),
          fake_n=("label", lambda x: (x == "fake").sum()),
          real_n=("label", lambda x: (x == "real").sum()),
      )
      .reset_index()
)

author_stats["fake_rate"] = author_stats["fake_n"] / author_stats["n"]
author_stats["real_rate"] = author_stats["real_n"] / author_stats["n"]

MIN_N = 5 # чтобы не ловить шум по авторам с 1-2 статьями
author_stats_f = author_stats[author_stats["n"] >= MIN_N].copy()

In [ ]:
author_stats_f.sort_values("fake_rate", ascending=False).head(15) # "фейковые" авторы

,author,n,fake_n,real_n,fake_rate,real_rate
6675,James Smith,7,6,1,0.857143,0.142857
11868,Michael Williams,5,4,1,0.800000,0.200000
4163,David Jones,5,4,1,0.800000,0.200000
9623,Kimberly Smith,5,4,1,0.800000,0.200000
11242,Matthew Smith,5,4,1,0.800000,0.200000
7836,John Brown,7,5,2,0.714286,0.285714
4126,David Garcia,6,4,2,0.666667,0.333333
11874,Michael Young,5,3,2,0.600000,0.400000
1328,Ashley Davis,5,3,2,0.600000,0.400000
11592,Michael Brown,5,3,2,0.600000,0.400000


In [ ]:
display(author_stats_f.sort_values("fake_rate", ascending=True).head(15)) # "правдивые" авторы

,author,n,fake_n,real_n,fake_rate,real_rate
1171,Anthony Jones,5,1,4,0.200000,0.800000
14242,Ryan Smith,5,1,4,0.200000,0.800000
7257,Jennifer Miller,5,1,4,0.200000,0.800000
5177,Elizabeth Johnson,5,1,4,0.200000,0.800000
7514,Jessica Brown,5,1,4,0.200000,0.800000
7978,John Smith,11,3,8,0.272727,0.727273
11718,Michael Lee,7,2,5,0.285714,0.714286
11828,Michael Smith,12,4,8,0.333333,0.666667
1412,Ashley Smith,6,2,4,0.333333,0.666667
8455,Joshua Smith,5,2,3,0.400000,0.600000


In [ ]:
# есть ли категории, где больше fake
ct = pd.crosstab(df["category"], df["label"], normalize="index")
ct["n"] = df["category"].value_counts()
ct = ct[ct["n"] >= 50].sort_values("fake", ascending=False)
ct

label,fake,real,n
category,,,
Business,0.516322,0.483678,2849
Health,0.507187,0.492813,2922
Entertainment,0.505365,0.494635,2889
Sports,0.503662,0.496338,2867
Politics,0.499286,0.500714,2802
Technology,0.494101,0.505899,2882
Science,0.493367,0.506633,2789


### Логистическая регрессия

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

In [ ]:
#TfidfVectorizer ожидает строки, поэтому мы просто склеиваем токены
df["text_for_model"] = df["tokens"].apply(lambda x: " ".join(x))

In [ ]:
tfidf = TfidfVectorizer(
    tokenizer=str.split, # используем готовые токены
    preprocessor=None,
    lowercase=False, # уже приведено к lower
    min_df=5,
    max_df=0.8,
    ngram_range=(1, 2), # биграммы
    sublinear_tf=True
)

In [ ]:
X = tfidf.fit_transform(df["text_for_model"])
y = (df["label"] == "fake").astype(int).values

D:\Python\Anaconda\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
feature_names = tfidf.get_feature_names_out()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [ ]:
clf = LogisticRegression(
    max_iter=3000,
    n_jobs=-1,
    class_weight="balanced"
)

clf.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=3000, n_jobs=-1)

In [ ]:
pred = clf.predict(X_test)
proba = clf.predict_proba(X_test)[:, 1]

print(classification_report(y_test, pred, target_names=["real", "fake"]))
print("ROC-AUC:", roc_auc_score(y_test, proba))
print("Confusion matrix:\n", confusion_matrix(y_test, pred))

              precision    recall  f1-score   support

        real       0.50      0.50      0.50      1989
        fake       0.51      0.51      0.51      2011

    accuracy                           0.51      4000
   macro avg       0.51      0.51      0.51      4000
weighted avg       0.51      0.51      0.51      4000

ROC-AUC: 0.5013519158954558
Confusion matrix:
 [[ 993  996]
 [ 979 1032]]


In [ ]:
coef = clf.coef_.ravel()

top_fake_idx = np.argsort(coef)[-20:][::-1]
top_real_idx = np.argsort(coef)[:20]

top_fake_words = [feature_names[i] for i in top_fake_idx]
top_real_words = [feature_names[i] for i in top_real_idx]

print("Топ-слов, которые толкают в FAKE:")
print(top_fake_words)

print("\nТоп-слов, которые толкают в REAL:")
print(top_real_words)

Топ-слов, которые толкают в FAKE:
['significant', 'particular', 'model', 'wrong', 'standard', 'far', 'fire', 'role', 'animal', 'site', 'claim', 'agency', 'rich', 'moment', 'modern', 'boy', 'trade', 'mother', 'seek', 'policy']

Топ-слов, которые толкают в REAL:
['economy', 'bring', 'big', 'reflect', 'girl', 'example', 'point', 'radio', 'play', 'dark', 'great', 'politic', 'sister', 'bar', 'weight', 'son', 'general', 'nice', 'congress', 'head']


Вывод: точность модели - рандом. Но учитывая, что EDA проводился достаточно скурпулёзный (несколькими способами), и это никак не влияло на качество модели существенно (всё равно рандом) + в ходе анализа визуально даже понятно, что фейковые и реальные новости очень тяжело различить, нет у них каких-то явных особенностей, зацепок, которые бы подсказывали. Наверное, это говорит о том, что из того, что нам дано в исходном датасете автоматизировать классификацию новостей на правду/фейк не представляется возможным (хорошо врут!).

# Выводы (девочки делали на занятии)

## 1. Предобработка текста
Для подготовки текстовых данных были выполнены следующие шаги:
- объединение заголовка и основного текста в единое текстовое поле;
- токенизация и лемматизация английского текста с использованием библиотеки **spaCy**;
- удаление пунктуации, чисел и стоп-слов;
- фильтрация слишком коротких токенов;
- формирование списка лемматизированных токенов для каждого документа.

Такая предобработка позволила привести тексты к нормализованному виду и снизить уровень шума.

---

## 2. Разведочный анализ (EDA)

### 2.1 Анализ по категориям
Для анализа лексических особенностей категорий был применён подход **class-specific TF-IDF**, при котором:
- TF рассчитывался внутри каждой категории,
- IDF — по всему корпусу.

Результаты показали, что в разных категориях часто встречаются схожие общеязыковые слова, характерные для новостного стиля в целом. Это указывает на слабую лексическую дифференциацию категорий на уровне отдельных слов.

### 2.2 Анализ fake / real
Аналогичный class-specific TF-IDF анализ для классов *fake* и *real* также показал значительное пересечение топ-слов между классами. Это означает, что:
- наиболее частотные и значимые слова корпуса являются общими для обоих классов,
- TF-IDF выявляет важные слова внутри класса, но не оптимизирован для поиска дискриминативных признаков между классами.

---

## 3. Анализ авторов и метаданных
Был выполнен анализ авторов с подсчётом количества публикаций и доли фейковых новостей (с фильтрацией по минимальному числу статей). Также была построена кросс-таблица `category × label`.

Полученные результаты не выявили устойчивых и однозначных закономерностей, позволяющих надёжно определять фейковость новости только по автору или категории, что характерно для учебных или синтетических датасетов.

---

## 4. Модель классификации

Для классификации была использована модель:
- **TF-IDF (scikit-learn)**
- **Logistic Regression**

Модель обучалась на текстовых признаках без использования метаданных.

### Результаты:
- ROC-AUC ≈ **0.50**
- Качество классификации соответствует уровню случайного угадывания.

Это свидетельствует о том, что в текущем датасете отсутствует сильная и устойчивая связь между текстовым содержанием и меткой fake / real, либо эта связь не выражена в используемом представлении текста.

---

## 5. Интерпретация результатов
Низкое качество модели не является ошибкой реализации. Напротив, полученные результаты позволяют сделать важные методологические выводы:
- модель не использует утечки данных и не «читерит» за счёт метаданных;
- текстовые признаки после предобработки не содержат достаточного сигнала для уверенной классификации.

---

## 6. Ограничения и возможные улучшения
- Проверка качества и происхождения разметки (учебный/синтетический характер данных).
- Ослабление фильтров предобработки текста и сравнение с baseline-моделью на сыром тексте.
- Применение более сложных моделей при наличии реальных, качественно размеченных данных.
